<a href="https://colab.research.google.com/github/sowmyarshetty/CapstoneProject-AI-Class/blob/main/AmazonHomeKitchenReviewsDataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import dask.dataframe as dd
import gdown
import re
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
# from bertopic import BERTopic
# from sentence_transformers import SentenceTransformer

In [4]:
# Mount Google Drive (For Colab Users)
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
metadatafileid = '1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0'
reviewsfileid = '1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5'
metafilename = 'meta_Home_and_Kitchen_50k.jsonl'
reviewsfilename = 'Home_and_Kitchen_1M.jsonl'

metadataurl = f"https://drive.google.com/uc?export=download&id={metadatafileid}"
reviewsurl = f"https://drive.google.com/uc?export=download&id={reviewsfileid}"

gdown.download(metadataurl,metafilename, quiet=False)
gdown.download(reviewsurl, reviewsfilename, quiet=False)



Downloading...
From (original): https://drive.google.com/uc?export=download&id=1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0
From (redirected): https://drive.google.com/uc?export=download&id=1eo8HyJeC5_G9KCHItFsLdJWLcCPTsrS0&confirm=t&uuid=f6bc4800-2f99-4a7f-999f-ec1cc0b9f3e0
To: /content/meta_Home_and_Kitchen_50k.jsonl
100%|██████████| 185M/185M [00:01<00:00, 125MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5
From (redirected): https://drive.google.com/uc?export=download&id=1LFUK3el3oLgjcl1WKy-MzalCHh8xnJk5&confirm=t&uuid=3dc6c020-ef14-4dc8-b3b6-da242d4aae8b
To: /content/Home_and_Kitchen_1M.jsonl
100%|██████████| 6.52G/6.52G [00:32<00:00, 200MB/s]


'Home_and_Kitchen_1M.jsonl'

* Read the json files into dataframes

In [6]:
df_meta = pd.read_json(metafilename,lines=True)
df_reviews = pd.read_json(reviewsfilename,lines=True)




*   Analyse the datasets




In [7]:
print(df_meta.columns)
print(df_reviews.columns)
pd.set_option("display.max_colwidth", None)
print(df_meta.head(1))

Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together'],
      dtype='object')
Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')
  main_category  \
0   Amazon Home   

                                                                                                                                             title  \
0  Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More   

   average_rating  rating_number  \
0             4.6             18   

                                                                                                                                                                                                        

* Extract the brand, color and price data from the details column of the metadata

In [13]:
df_meta_normalized = df_meta
df_metadetails = pd.json_normalize(df_meta_normalized["details"])
# df_meta_normalized = df_meta_normalized.drop(columns=["details"]).join(df_metadetails)  # Merge with main DataFrame
df_meta_images = pd.json_normalize(df_meta_normalized["images"])
# print(df_metadetails.columns)
# print(df_meta_images.columns)
df_meta_normalized = df_meta_normalized.drop(columns=["details","images","videos"]).join(df_metadetails[["Brand","Material","Color","Style","Pattern"]])
# df_meta_normalized = df_meta_normalized.drop(columns=["images"]).join(df_meta_images[["main_image"]])
print(df_meta_normalized.columns)

In [14]:
print(df_meta_normalized.head(1))

  main_category  \
0   Amazon Home   

                                                                                                                                             title  \
0  Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More   

   average_rating  rating_number  \
0             4.6             18   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         features  \
0  [☕PERFECT IRISH COFFEE MUG: With ou

* Merge the data sets from the meta and reviews with the relevant columns with parent_asin as the join key and index

In [15]:
df_merged = df_reviews.merge(df_meta_normalized[['title','description','price','Brand','Material','Color','categories',"parent_asin"]],on='parent_asin',how='left')

* Look for reviews with missing product metadata

In [16]:
missing_asins_df = df_reviews[~df_reviews['parent_asin'].isin(df_meta_normalized['parent_asin'])]
print(missing_asins_df[['parent_asin']].drop_duplicates().count())
print(missing_asins_df[['parent_asin']].count())
print(len(df_reviews))
print(len(df_merged))
print(df_merged.columns)

parent_asin    1731834
dtype: int64
parent_asin    12359277
dtype: int64
13376345
13376345
Index(['rating', 'title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'title_y',
       'description', 'price', 'Brand', 'Material', 'Color', 'categories'],
      dtype='object')


* Remove the reviews which do not have product metadata

In [17]:
df_new = df_merged.dropna(subset = ['title_y','description','price','Brand','Material','Color','categories','parent_asin'])
print(len(df_new))

754079


In [18]:
df_new.to_csv('AmazonHomeKitchenReviews.csv')

In [19]:
 !cp /content/AmazonHomeKitchenReviews.csv /content/drive/MyDrive/